### Decision Trees - Regression (numpy)

#### Decision Trees
Decision Trees are predictive classifiction algorithms which split (branch) training data in a way that they can reach a conclusion (leaf). The tree is then used to predict the values using the test dataset.<br>
There are different 'ways' this trees 'decide' where and in what order to split (here are a couple of them):<br>
 - Gini impurity: very usefull when dataset has labels
 - Variance reduction: used when labels take the form of continuous numbers

Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees.<br>

#### Regression Trees
Decision trees where the prediction label are real numbers.<br>

__In the following implementation we will be using only numpy.__

The data was obtained from:<br>
__Dua, D. and Graff, C. (2019). UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Car+Evaluation). Irvine, CA: University of California, School of Information and Computer Science.__<br>

Abstract: The model evaluates cars according to the following concept structure:<br>


__Class Values__: unacc, acc, good, vgood 

__Attributes:__<br>
__buying__: vhigh, high, med, low.<br>
__maint__: vhigh, high, med, low<br>
__doors__: 2, 3, 4, 5more<br>
__persons__: 2, 4, more<br>
__lug_boot__: small, med, big<br>
__safety__: low, med, high<br>

In [71]:
import csv
import numpy as np

#### Loading Data

In [72]:
file='car.data'
with open(file, 'r') as csvfile:
    rows=[]
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        rows.append(row)
    print("Total no. of rows: %d"%(len(rows)))

Total no. of rows: 1728


#### Data Transformation
We are going to transform all labels and text to numeric data in order to use regression trees.

In [73]:
n=len(rows)
d=len(rows[0])

for i in range(d):
    field_values=set()
    for row in rows:
        field_values.add(row[i])
    field_values
    print('Field:',i,'Values:',field_values)

Field: 0 Values: {'high', 'low', 'vhigh', 'med'}
Field: 1 Values: {'high', 'low', 'vhigh', 'med'}
Field: 2 Values: {'3', '2', '5more', '4'}
Field: 3 Values: {'2', '4', 'more'}
Field: 4 Values: {'big', 'small', 'med'}
Field: 5 Values: {'high', 'low', 'med'}
Field: 6 Values: {'good', 'acc', 'vgood', 'unacc'}


In [74]:
dict_0 = {'high':2, 'low':0, 'med':1, 'vhigh':3}  #we can use this dictionary for columns 0, 1 and 5
dict_2 = {'5more':5, '3':3, '2':2, '4':4}             #dictionary for column 2
dict_3 = {'2':2, 'more':5, '4':4}                     #dictionary for column 3
dict_4 = {'big':2, 'small':0, 'med':1}                #dictionary for column 4
dict_6 = {'good':2, 'acc':1, 'unacc':0, 'vgood':3}    #dictionary for column 6

dataset_list=[]
for row in range(n):
    temp=[]
    for column in range(d):
        if column in [0,1,5]:
            temp.append(dict_0[rows[row][column]])
        elif column ==2:
            temp.append(dict_2[rows[row][column]])
        elif column == 3:
            temp.append(dict_3[rows[row][column]])
        elif column == 4:
            temp.append(dict_4[rows[row][column]])
        elif column == 6:
            temp.append(dict_6[rows[row][column]])
        else:
            temp.append('ERROR')
    dataset_list.append(temp)

for i in range(d):
    field_values=set()
    for row in dataset_list:
        field_values.add(row[i])
    field_values
    print('Field:',i,'Values:',field_values)

Field: 0 Values: {0, 1, 2, 3}
Field: 1 Values: {0, 1, 2, 3}
Field: 2 Values: {2, 3, 4, 5}
Field: 3 Values: {2, 4, 5}
Field: 4 Values: {0, 1, 2}
Field: 5 Values: {0, 1, 2}
Field: 6 Values: {0, 1, 2, 3}


#### Creating Training and Testing Datasets

In [75]:
dataset = np.array(dataset_list)

X=dataset[:,0:d-1]                         #first columns for X
y=dataset[:,-1]                            #last column is y (label)

indices = list(np.random.permutation(n))   #indices in random order
train_proportion = 0.7                     #desired proportion for training dataset
train_size=int(train_proportion*n)

train_indices = indices[0:train_size]      #train indices
test_indices = indices[train_size:]        #test indices

print('Proportion of data for training:',int((10000*len(train_indices)/(len(train_indices)+len(test_indices))))/100,'%')

#Train and Test Datasets
X_train = X[train_indices,:]
y_train = y[train_indices]
X_test = X[test_indices,:]
y_test = y[test_indices]

Proportion of data for training: 69.96 %


In [76]:
def variance(y):
    #This function will calculate how much variance exists between y labels
    #Input:
    #  y: numeric labels that will be used for evaluating variance
    #Output:
    #  variance: calculates variance of labels
    return np.sum((y - np.mean(y))**2)

In [77]:
def sqsplit(X_train, y_train):
    #This function will evaluate where to split using the variance function
    #Input:
    #    X_train: X training set (numpy)
    #    y_train: y training set (label vector)
    #Output:
    #    feature:  or column where data splits
    #    cut: cut-value of the best cut
    #    bestvar: variance where best cut happens
    
    N,D = X_train.shape
    bestvar = np.inf
    feature = np.inf
    cut = np.inf
    for i in range(D):
        ordered_indices = np.argsort(X_train[:,i])        
        for j in range(N-1):
            var =  variance( y_train[ordered_indices[0:j+1]] ) + variance( y_train[ordered_indices[j+1:]] )
            if var <= bestvar and X_train[ordered_indices[j+1],i] != X_train[ordered_indices[j],i]:
                bestvar = var
                cut = ( X_train[ordered_indices[j+1],i] + X_train[ordered_indices[j],i] ) / 2
                feature = i
    return feature, cut, bestvar

In [78]:
class TreeNode(object):
    #Tree class: we will be saving tree/branch/leaf information
    #
    def __init__(self, left, right, feature, cut, prediction):
        self.left = left
        self.right = right
        self.feature = feature
        self.cut = cut
        self.prediction = prediction

def cart(X_train,y_train):
    #CART Tree: we will be using this function recursively. We will first put an if to see if we need to stop 
    #  constructing the tree or we should continue creating more branches/leaves
    #Input:
    #  X_train: feature matrix
    #  y_train: label vector
    #Output:
    #  tree: decision tree

    n,d = X_train.shape
    indices_in_range_n = np.arange(n)
    prediction = np.mean(y_train)
    #we will stop if all y-labels are the same
    if np.all(y_train == y_train[0]):
        return TreeNode(None, None, None, None, prediction)    #this would be a leaf
    else:
        #this would be a branch
        feature, cut, bestvar = sqsplit(X_train, y_train)                    #find new place where to split
        left_indices =  indices_in_range_n[X_train[:, feature] <= cut]       #left side
        right_indices =  indices_in_range_n[X_train[:, feature] > cut]       #right side
        left_leaf = cart(X_train[left_indices,:],y_train[left_indices])      #find new branch/leaf
        right_leaf = cart(X_train[right_indices,:],y_train[right_indices])   #find new branch/leaf
        tree = TreeNode(left_leaf, right_leaf, feature, cut, prediction)     #construct the tree
        left_leaf.parent = tree
        right_leaf.parent = tree
        return tree

def predictions(tree,X_test):
    #find predictions following branches of the decision tree
    #Input:
    #  tree: TreeNode decision tree
    #  X_test:  n x d matrix of data points
    #Output:
    #  pred: n-dimensional vector of predictions

    n,d = X_test.shape
    pred = np.zeros(n)    #empty prediction vector

    for i in range(n):
        current_node = tree
        while True:                 #infinite loop until it reaches a leaf
            if current_node.left == current_node.right == None:   #if both sides are None it is a leaf
                pred[i] = current_node.prediction
                break               #found solution, so break to continue with next i
            else:
                if X_test[i,current_node.feature] <= current_node.cut:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
    return pred

In [79]:
parent=cart(X_train,y_train)
prediction=predictions(parent,X_test)

print('Accuracy:',round(100*np.sum(prediction==y_test)/len(prediction),2))

Accuracy: 97.69
